In [1]:
import theano

Using cuDNN version 7103 on context None
Mapped name None to device cuda0: GeForce GTX 1080 (0000:02:00.0)


In [2]:
%run Toolkit_particle.ipynb

/home/solved/.virtualenvs/MEMDK/local/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


Welcome to JupyROOT 6.12/06


In [ ]:
%run Toolkit_utils.ipynb

Using TensorFlow backend.


In [ ]:
%run Plot_distributions.ipynb

In [ ]:
import cPickle as pickle
import logging
import os
import sys
import json

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 10)

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
import deepdish.io as io
from tqdm import tqdm
import keras
from tqdm import tqdm
import pandautils as pup

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

#Changed to notebooks:
#from toolkit.particles import Jet, Lepton
#from toolkit.utils import parallel_run, safe_mkdir
#from plots import plot_distributions

In [ ]:
# -- logging stuff
LOGGER_PREFIX = ' %s'
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
def log(msg):
    logger.info(LOGGER_PREFIX % msg)

In [ ]:
TREE = 'METree'
#ONLY_4V = False
# Each one of the root files has one TTree with 38 branches (variables). Out of those, we pick these:
# -- select variables needed in this analysis, including inputs and cuts
signal_vars = ['ME_Sig_Obj_pt', 'ME_Sig_Obj_eta', 'ME_Sig_Obj_phi', 'ME_Sig_Obj_E',
              'ME_Sig_Likelihoods_Available',
              'ME_Sig_Mod_Likelihoods',
              'ME_Sig_Obj_type'] 
bkg_vars = ['ME_Bkg_Mod_Likelihoods', 'ME_Bkg_Likelihoods_Available']

In [ ]:
#data ="type of data to use ('1d', '7d', or 'reco')". Replaces args.data in python code.
data = '1d'

SAMPLE_PATH = os.path.join('../data', data)
safe_mkdir(SAMPLE_PATH)

higgs_signal_files = sorted([f for f in os.listdir(SAMPLE_PATH) 
    if (('Sig' in f) and (('ttH' in f)))])
higgs_bkg_files = sorted([f for f in os.listdir(SAMPLE_PATH) 
    if (('Bkg' in f) and (('ttH' in f)))])
ttbar_signal_files = sorted([f for f in os.listdir(SAMPLE_PATH) 
    if (('Sig' in f) and ('bjet' in f) and ('rfast' not in f) and ('PPP' not in f))])
ttbar_bkg_files = sorted([f for f in os.listdir(SAMPLE_PATH) 
    if (('Bkg' in f) and ('bjet' in f) and ('rfast' not in f) and ('PPP' not in f))])

print 'hs:', higgs_signal_files,'\n'
print 'hb:', higgs_bkg_files,'\n'
print 'ts:', ttbar_signal_files,'\n'
print 'tb:', ttbar_bkg_files,'\n'

In [ ]:

log('loading signal ROOT files')
# -- load signal samples
higgs_signal_df = pup.root2panda(
    [os.path.join(SAMPLE_PATH, signal) for signal in higgs_signal_files], TREE,
    branches = signal_vars
)
print [os.path.join(SAMPLE_PATH, signal) for signal in higgs_signal_files]
ttbar_signal_df = pup.root2panda(
    [os.path.join(SAMPLE_PATH, signal) for signal in ttbar_signal_files], TREE,
    branches = signal_vars
)
print  [os.path.join(SAMPLE_PATH, signal) for signal in ttbar_signal_files]
log('loading background ROOT files')
# -- load background samples
higgs_bkg_df = pup.root2panda(
    [os.path.join(SAMPLE_PATH, bkg) for bkg in higgs_bkg_files], TREE,
    branches = bkg_vars
)
print [os.path.join(SAMPLE_PATH, bkg) for bkg in higgs_bkg_files]
ttbar_bkg_df = pup.root2panda(
    [os.path.join(SAMPLE_PATH, bkg) for bkg in ttbar_bkg_files], TREE,
    branches = bkg_vars
)
print  [os.path.join(SAMPLE_PATH, bkg) for bkg in ttbar_bkg_files]
log('applying preselections')
higgs_slice = np.logical_and(
    higgs_signal_df['ME_Sig_Likelihoods_Available'] == True, 
    higgs_bkg_df['ME_Bkg_Likelihoods_Available'] == True
    )
ttbar_slice = np.logical_and(
    ttbar_signal_df['ME_Sig_Likelihoods_Available'] == True, 
    ttbar_bkg_df['ME_Bkg_Likelihoods_Available'] == True
    )

In [ ]:
unsorted_dataframe = pd.DataFrame(data=higgs_signal_df)[::]


In [ ]:
unsorted_dataframe

In [ ]:
# build X from those 4 branches only (pt, eta, phi, e)
    # use the objects from position 2 to 10
    # position 0 and 1 are the initial state objects, with type == 0
    # position 2 to 5 are b jets
    # position 6 and 7 are light jets
    # position 8 and 9 are leptons and MET
    # position 10+ are spectator jets



#bjets--> 2 to 5
#light jets --> 6 and 7
#rest--> 8,9,10

In [ ]:
#no_spectators replaces args.no_spectators
no_spectators = False
# -- to remove spectator jets
if no_spectators is True:
    higgs_slice = np.logical_and(
        higgs_slice, 
        np.array([(-1 not in event) for event in higgs_signal_df["ME_Sig_Obj_type"]])
        )
    ttbar_slice = np.logical_and(
        ttbar_slice, 
        np.array([(-1 not in event) for event in ttbar_signal_df["ME_Sig_Obj_type"]])
        )
    
higgs_signal_df_nospec = higgs_signal_df[higgs_slice]
ttbar_signal_df_nospec = ttbar_signal_df[ttbar_slice]
higgs_bkg_df_nospec = higgs_bkg_df[higgs_slice]
ttbar_bkg_df_nospec = ttbar_bkg_df[ttbar_slice]


In [ ]:

log('applying lepton pt cut')
# indices of Rows To Be Removed
rtbr = [n for n, ev in ttbar_signal_df.iterrows() if ev['ME_Sig_Obj_pt'][np.logical_or(
        ev['ME_Sig_Obj_type'] == 11,
        ev['ME_Sig_Obj_type'] == 13)] < 20]

print 'rtbr', rtbr
ttbar_signal_df.drop(rtbr, inplace=True)
ttbar_bkg_df.drop(rtbr, inplace=True)


In [ ]:
#-----
def build_soft_target(sig_df, bkg_df, alpha=0.1):
    ME_Bkg = np.array([a[0] for a in bkg_df['ME_Bkg_Mod_Likelihoods']])
    ME_Sig = np.array([a[0] for a in sig_df['ME_Sig_Mod_Likelihoods']])
    soft_target = ME_Sig / (ME_Sig + alpha * ME_Bkg)
    return soft_target


print 'ttbar_signal_df.shape', ttbar_signal_df.shape
print 'ttbar_signal_df type', type(ttbar_signal_df)

print 'ttbar_bkg_df.shape', ttbar_bkg_df.shape
print 'ttbar_bkg_df type', type(ttbar_bkg_df)



log('building soft_target')
bkg_llh = np.log(np.array(
    [a[0] for a in higgs_bkg_df['ME_Bkg_Mod_Likelihoods']] + [a[0] for a in ttbar_bkg_df['ME_Bkg_Mod_Likelihoods']]
))
signal_llh = np.log(np.array(
    [a[0] for a in higgs_signal_df['ME_Sig_Mod_Likelihoods']] + [a[0] for a in ttbar_signal_df['ME_Sig_Mod_Likelihoods']]
))
if data == 'reco':
    alpha = 0.1
else:
    alpha = 0.001
soft_target = np.array(
    build_soft_target(higgs_signal_df, higgs_bkg_df, alpha).tolist() + 
    build_soft_target(ttbar_signal_df, ttbar_bkg_df, alpha).tolist())

del higgs_bkg_df, ttbar_bkg_df
#-----

In [ ]:
log('building y')
y = np.array(([1] * higgs_signal_df.shape[0]) + ([0] * ttbar_signal_df.shape[0]))


In [ ]:
log('building X')
df = pd.concat([higgs_signal_df, ttbar_signal_df], ignore_index=True)
print 'df shape', np.shape(df)

# build X from those 4 branches only (pt, eta, phi, e)
# use the objfgects from position 2 to 10
# position 0 and 1 are the initial state objects, with type == 0
# position 2 to 5 are b jets
# position 6 and 7 are light jets
# position 8 and 9 are leptons and MET
# position 10+ are spectator jets
X = np.array([np.asarray(row.tolist())[:, 2:10].ravel() 
    for row in tqdm(df[['ME_Sig_Obj_pt', 'ME_Sig_Obj_eta', 'ME_Sig_Obj_phi', 'ME_Sig_Obj_E']].values)])
X_full = X
print X     
  

In [ ]:
df

In [ ]:
log('building X')
df = pd.concat([higgs_signal_df, ttbar_signal_df], ignore_index=True)
print 'df shape', np.shape(df)

# build X from those 4 branches only (pt, eta, phi, e)
# use the objects from position 2 to 10
# position 0 and 1 are the initial state objects, with type == 0
# position 2 to 5 are b jets
# position 6 and 7 are light jets
# position 8 and 9 are leptons and MET
# position 10+ are spectator jets
    for row in tqdm(df[['ME_Sig_Obj_pt', 'ME_Sig_Obj_eta', 'ME_Sig_Obj_phi', 'ME_Sig_Obj_E']].values)])
X_full = X
print X     
  

In [ ]:
log('building the four vector')
#df = pd.concat([higgs_signal_df, ttbar_signal_df], ignore_index=True)
#print 'df shape', np.shape(df)
#print df
four_vector_and_type=df.loc[:,['ME_Sig_Obj_pt', 'ME_Sig_Obj_eta','ME_Sig_Obj_phi','ME_Sig_Obj_E','ME_Sig_Obj_type']]

four_vector = four_vector_and_type.loc[:,'ME_Sig_Obj_pt':'ME_Sig_Obj_E']



In [ ]:
four_vector


In [ ]:
sortable_cols = ['ME_Sig_Obj_pt', 'ME_Sig_Obj_eta', 'ME_Sig_Obj_phi', 'ME_Sig_Obj_E', 'ME_Sig_Obj_type']
def sortby(column):
    unsorted_dataframe = four_vector_and_type.DataFrame(data=higgs_signal_df)[::]
    unsorted_dataframe = unsorted_dataframe[0:5]
    sorted_dataframe = unsorted_dataframe[::]
    for col in sortable_cols:
        for n in range(len(unsorted_dataframe)):
            sorter = np.concatenate(0,1,(sorted_by_pt(n)+2))
            print sorter
            col_position = higgs_signal_df.columns.get_loc(hcol)
            sorted_col = [[higgs_signal_df.iloc[n,col_position][sorter[::]]]]
            sorted_dataframe.iloc[n,col_position] = sorted_col[::]
    return sorted_dataframe

In [ ]:
sortable_cols = ['ME_Sig_Obj_pt', 'ME_Sig_Obj_eta', 'ME_Sig_Obj_phi', 'ME_Sig_Obj_E', 'ME_Sig_Obj_type']
def sortby(column):
    unsorted_dataframe = four_vector.DataFrame(data=higgs_signal_df)[::]
    sorted_dataframe = unsorted_dataframe[::]
    for col in sortable_cols:
        #for n in tqdm(range(len(unsorted_dataframe))):
        for n in tqdm(range(2)):
            print n
            sorter = (sorted_by_pt(n))[1]
            print sorter
            new_sorter = np.append((0,1),sorter+2)
            print new_sorter
            col_position = unsorted_dataframe.columns.get_loc(col)
            print col_position
            col_to_sort = unsorted_dataframe.iloc[n,col_position]
            print col_to_sort
            sorted_col = col_to_sort[new_sorter]
            print sorted_col
            print np.shape(unsorted_dataframe)
            print sorted_dataframe
            print sorted_dataframe.iloc[n,col_position]
            print sorted_col
            print np.shape(sorted_dataframe.iloc[n,col_position])
            print np.shape(sorted_col)
            entries= len(sorted_col)
            print sorted_dataframe.iloc[n,col_position][:20]
            sorted_dataframe.iloc[n,col_position][:entries] = sorted_col[:]
    return sorted_dataframe


In [ ]:
sortable_cols = ['ME_Sig_Obj_pt', 'ME_Sig_Obj_eta', 'ME_Sig_Obj_phi', 'ME_Sig_Obj_E', 'ME_Sig_Obj_type']
def sortby(column):
    unsorted_dataframe = pd.DataFrame(data=higgs_signal_df)[::]
    sorted_dataframe = unsorted_dataframe[::]
    for col in sortable_cols:
        #for n in tqdm(range(len(unsorted_dataframe))):
        for n in tqdm(range(2)):
            print n
            sorter = (sorted_by_pt(n))[1]
            print sorter
            new_sorter = np.append((0,1),sorter+2)
            print new_sorter
            col_position = unsorted_dataframe.columns.get_loc(col)
            print col_position
            col_to_sort = unsorted_dataframe.iloc[n,col_position]
            print col_to_sort
            sorted_col = col_to_sort[new_sorter]
            print sorted_col
            print np.shape(unsorted_dataframe)
            print sorted_dataframe
            print sorted_dataframe.iloc[n,col_position]
            print sorted_col
            print np.shape(sorted_dataframe.iloc[n,col_position])
            print np.shape(sorted_col)
            entries= len(sorted_col)
            print sorted_dataframe.iloc[n,col_position][:20]
            sorted_dataframe.iloc[n,col_position][:entries] = sorted_col[:]
    return sorted_dataframe


In [ ]:
sortby('ME_Sig_Obj_pt')

In [ ]:
def replacing_column(unsorted_dataframe,replacement,position):
    sorted_dataframe = unsorted_dataframe[::]
    for n in tqdm(range(len(sorted_dataframe))):
        entries= len(sorted_col)
        sorted_dataframe.iloc[n,position][:entries] = sorted_col[:]


    

In [ ]:
unsorted_dataframe = pd.DataFrame(data=higgs_signal_df)[::]
pt_column='ME_Sig_Obj_pt'
replacing_column(unsorted_dataframe,new_col,pt_column)

In [ ]:
  #log('plotting unscaled input variables')
    #safe_mkdir(os.path.join('plots', data))
    #plot_distributions(X_train, y_train, soft_target_train, bkg_llh_train, signal_llh_train,\
    #    X_test, y_test, soft_target_test, bkg_llh_test, signal_llh_test,\
    #    X_eval, y_eval, soft_target_eval, bkg_llh_eval, signal_llh_eval, '_unscaled', data)
    #print 'unscaled input variables plotted'

    scaler = RobustScaler(quantile_range=(25, 75))
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_eval = scaler.transform(X_eval)

    #log('plotting scaled input variables')
    #plot_distributions(X_train, y_train, soft_target_train, bkg_llh_train, signal_llh_train,\
    #    X_test, y_test, soft_target_test, bkg_llh_test, signal_llh_test,\
    #    X_eval, y_eval, soft_target_eval, bkg_llh_eval, signal_llh_eval, '_scaled', data)
    #print 'scaled input variables plotted'


In [ ]:

    # -- save scaling parameters to be used by lwtnn converter
    log('saving scaling options to ' + os.path.join(SAMPLE_PATH, 'variables.json'))
    var_names = [
        'pT_b-jet_1', 'pT_b-jet_2', 'pT_b-jet_3', 'pT_b-jet_4', 'pT_l-jet_1', 'pT_l-jet_2', 'pT_lepton', 'pT_MET', 
        'eta_b-jet_1', 'eta_b-jet_2', 'eta_b-jet_3', 'eta_b-jet_4', 'eta_l-jet_1', 'eta_l-jet_2', 'eta_lepton', 'eta_MET',
        'phi_b-jet_1', 'phi_b-jet_2', 'phi_b-jet_3', 'phi_b-jet_4', 'phi_l-jet_1', 'phi_l-jet_2', 'phi_lepton', 'phi_MET',
        'E_b-jet_1', 'E_b-jet_2', 'E_b-jet_3', 'E_b-jet_4', 'E_l-jet_1', 'E_l-jet_2', 'E_lepton', 'E_MET']
    # -- write variable json for lwtnn keras2json converter
    variable_dict = {
        'inputs': [{
            'name': var_names[v],
            'scale': float(1.0 / scaler.scale_[v]),
            #'offset': float(-scaler.mean_[v]),
            'default': None
            } 
            for v in xrange(len(var_names))],
        'class_labels': ['bkg_llh', 'signal_llh'],
        'keras_version': keras.__version__
    }
    with open(os.path.join(SAMPLE_PATH, 'variables.json'), 'wb') as varfile:
        json.dump(variable_dict, varfile)


In [ ]:

    # -- save data archives
    train_ = {
        'X' : X_train,
        'y' : y_train,
        #'soft_target' : soft_target_train,
        #'bkg_llh' : bkg_llh_train,
        'signal_llh' : signal_llh_train
    }

    test_ = {
        'X' : X_test,
        'y' : y_test,
        #'soft_target' : soft_target_test,
        #'bkg_llh' : bkg_llh_test,
        'signal_llh' : signal_llh_test
    }

    eval_ = {
        'X' : X_eval,
        'y' : y_eval,
        #'soft_target' : soft_target_eval,
        #'bkg_llh' : bkg_llh_eval,
        'signal_llh' : signal_llh_eval
    }

    log('saving to h5 archive...')
    spectators = '_NO_SPECTATORS' if no_spectators else ''
    io.save(os.path.join(SAMPLE_PATH, 'MEM-A-0500' + spectators + '.h5'), train_)
    io.save(os.path.join(SAMPLE_PATH, 'MEM-B-0500' + spectators +'.h5'), test_)
    io.save(os.path.join(SAMPLE_PATH, 'MEM-C-0500' + spectators +'.h5'), eval_)
    print (os.path.join(SAMPLE_PATH, 'MEM-A-0500' + spectators + '.h5'), train_)
    print (os.path.join(SAMPLE_PATH, 'MEM-B-0500' + spectators +'.h5'), test_)
    print (os.path.join(SAMPLE_PATH, 'MEM-C-0500' + spectators +'.h5'), eval_)

In [ ]:




    # def shape_matched_from_df(cls, df, ids):
    #     t = df.shape[0] - 1
    #     particles = []
    #     # for i, (_, p) in enumerate(df[ids].iterrows()):
    #     for i, p in enumerate(zip(*[df[_id].values for _id in ids])):
    #         if i % 1000 == 0:
    #             sys.stdout.write('\r{:.2f}% complete.'.format(100 * (float(i) / t)))
    #             sys.stdout.flush()
    #         particles.append([cls(*v) for v in zip(*p)])
    #     sys.stdout.write('\nDone.')
    #     return particles
    #     # return pup.match_shape(np.array(particles), df[ids[0]])




    # log('Starting particle matching using wrapper classes.')
    # # -- build shape matched lists of particles...

    # jet_features = ['jet_pt', 'jet_eta', 'jet_phi', 'jet_E']
    # lepton_features = ['lep_pt', 'lep_eta', 'lep_phi', 'lep_E']

    # log('Shape matching background jets...')
    # bkg_jets = shape_matched_from_df(Jet, bkg_df, jet_features)

    # log('Shape matching background leptons...')
    # bkg_leps = shape_matched_from_df(Lepton, bkg_df, lepton_features)

    # log('Shape matching signal jets...')
    # signal_jets = shape_matched_from_df(Jet, signal_df, jet_features)

    # log('Shape matching signal leptons...')
    # signal_leps = shape_matched_from_df(Lepton, signal_df, lepton_features)


    # log('Loading old dataframes')
    # # -- load in pre-saved dataframes with all other variables
    # old_bkg_df = pd.read_pickle(os.path.join(SAMPLE_PATH, 'bkg_df.pkl'))
    # old_signal_df = pd.read_pickle(os.path.join(SAMPLE_PATH, 'signal_df.pkl'))

    # log('Adding energy to old dataframes')
    # -- add the jet energy to the dataframe
    # old_bkg_df['jet_E'] = bkg_df['jet_E']
    # old_signal_df['jet_E'] = signal_df['jet_E']

    # -- add the jet and lepton 4-vectors to the dataframe
    # log('Adding newly computed quantities to old dataframes')
    # bkg_df['jet_4v'] = bkg_jets
    # signal_df['jet_4v'] = signal_jets
    # bkg_df['lep_4v'] = bkg_leps
    # signal_df['lep_4v'] = signal_leps

    # # -- get some truth up in here
    # bkg_df['truth'] = 0
    # signal_df['truth'] = 1

    # # -- get the permutations
    # log('Grabbing background permutations')
    # bkg_file = ROOT.TFile(os.path.join(SAMPLE_PATH, BKG_FILE_PATTERN))

    # bkg_tree = bkg_file.Get(BKG_TREE)

    # bkg_perm = []
    # nb_entries = bkg_tree.GetEntries()

    # log('fetching permutations')

    # for i in xrange(nb_entries):
    #     bkg_tree.GetEntry(i)
    #     sp = []
    #     for j in xrange(bkg_tree.ME_Sig_Mod_Permutations[0][0].size()):
    #         sp.append(int(bkg_tree.ME_Sig_Mod_Permutations[0][0][j]))
    #     bkg_perm.append(sp[:-2])

    # bkg_df['jet_order'] = bkg_perm

    # log('Grabbing signal permutations')
    
    # signal_file = ROOT.TFile(os.path.join(SAMPLE_PATH, SIGNAL_FILE_PATTERN))

    # signal_tree = signal_file.Get(SIGNAL_TREE)

    # signal_perm = []
    # nb_entries = signal_tree.GetEntries()

    # log('fetching permutations')

    # for i in xrange(nb_entries):
    #     signal_tree.GetEntry(i)
    #     sp = []
    #     for j in xrange(signal_tree.ME_Sig_Mod_Permutations[0][0].size()):
    #         sp.append(int(signal_tree.ME_Sig_Mod_Permutations[0][0][j]))
    #     signal_perm.append(sp[:-2])

    # signal_df['jet_order'] = signal_perm

    # # -- get the weights
    # log('Grabbing event weights')
    # bkg_df['evt_weight'] = [a[4] * b * c for (a, b, c) in zip(bkg_df['TRFMCweight_in'], bkg_df['ttbarTopPtDataWeight'], bkg_df['LeptonSF'])]
    # signal_df['evt_weight'] = [(a[4] * b) / c for (a, b, c) in zip(signal_df['TRFMCweight_in'], signal_df['LeptonSF'], signal_df['ttH_NLO_TruthWeight'])]

    # log('applying preselections')
    # # -- cut according to Olaf's instructions
    # bkg_cut = (bkg_df['ME_Bkg_Likelihoods_Available'] == 1) & \
    #           (bkg_df['ME_Sig_Likelihoods_Available'] == 1)

    # signal_cut = (signal_df['ME_Bkg_Likelihoods_Available'] == 1) & \
    #              (signal_df['ME_Sig_Likelihoods_Available'] == 1)

    # log('concatenating signal and background')
    # # -- concatenate signal and background df's
    # df = pd.concat(
    #     [bkg_df[bkg_cut], signal_df[signal_cut]], 
    #     ignore_index=True
    # )

    # # need to reorder jets according to the analysis's scheme
    # # first 4 are b-tagged, other 2 are leading untagged jets
    # # this will give you the indices

    # log('getting reordering')
    # reordering = np.array(df['jet_order'])

    # log('reordering particles')
    # df['ordered_jet_4v'] = [np.array(p)[ix].tolist() for p, ix in zip(df['jet_4v'], reordering)]

    # # -- list of dot products among jets (21 per event)
    # log('Generating dot products...')
    # dot_prod_list = [
    #                     [
    #                         (a.lv + b.lv).M() #a.lv.Dot(b.lv) 
    #                         for i, a in enumerate(e) 
    #                             for j, b in enumerate(e) 
    #                                 if j >= i
    #                     ] 
    #                     for e in df['ordered_jet_4v']
    #                 ]



    # log('Making X and y')

    # y = np.array(df['truth'])

    # N_COL = 23 

    # if not ONLY_4V:
    #     N_COL = N_COL + 21 + 5  # regular variables + dot products + other features

    # X = np.zeros((df.shape[0], N_COL))

    # log('adding lepton info')

    # X[:, 0] = df['lep_pt'];
    # X[:, 0] = np.log(X[:, 0]) 
    # X[:, 1] = df['lep_eta'];
    # X[:, 2] = df['lep_phi'];
    # X[:, 3] = df['metx'];
    # X[:, 4] = df['mety'];


    # log('adding jet info')
    # PT_COLUMN = 5
    # N_JETS = 6

    # ETA_COLUMN = PT_COLUMN + N_JETS
    # PHI_COLUMN = ETA_COLUMN + N_JETS
           
    # X[:, PT_COLUMN : (PT_COLUMN + N_JETS)] = [np.log(np.array(e))[ix].tolist() for e, ix in zip(df['jet_pt'], reordering)]

    # X[:, ETA_COLUMN : (ETA_COLUMN + N_JETS)] = [np.array(e)[ix].tolist() for e, ix in zip(df['jet_eta'], reordering)]

    # X[:, PHI_COLUMN : (PHI_COLUMN + N_JETS)] = [np.array(e)[ix].tolist() for e, ix in zip(df['jet_phi'], reordering)]


    # if not ONLY_4V:
    #     log('adding dot products')
    #     # -- add the dot products...
    #     DOT_COLUMN = PHI_COLUMN + N_JETS
    #     FEATURE_COLUMN = (DOT_COLUMN + 21)

    #     X[:, DOT_COLUMN : FEATURE_COLUMN] = np.array(dot_prod_list)




    #     log('adding secondary features')
    #     # -- angle between the first and second b jets
    #     X[:, FEATURE_COLUMN] = [e[0].lv.Angle(e[1].lv) for e in df['ordered_jet_4v']]

    #     # -- angle between the light jets
    #     X[:, FEATURE_COLUMN + 1] = [e[4].lv.Angle(e[5].lv) for e in df['ordered_jet_4v']]

    #     # -- decay angle of the first jet
    #     X[:, FEATURE_COLUMN + 2] = [e[0].lv.CosTheta() for e in df['ordered_jet_4v']]

    #     # -- mass of reconstructed bb+qq+b system (hopefully similar to top mass?)
    #     X[:, FEATURE_COLUMN + 3] = [(e[0].lv + e[1].lv + e[2].lv + e[4].lv + e[5].lv).M() / 1000 for e in df['ordered_jet_4v']]

    #     # -- qq product, should find W mass...
    #     X[:, FEATURE_COLUMN + 4] = [(e[4].lv + e[5].lv).M() for e in df['ordered_jet_4v']]

    # # -- hard target
    # y = df['truth'].values

    # log('scaling data')
    # scaler = StandardScaler()
    # Z = scaler.fit_transform(np.array(X))

    # soft_target = (df['ME_Sig_Mod_Likelihoods'] / (df['ME_Sig_Mod_Likelihoods'] + 0.23 * df['ME_Bkg_Mod_Likelihoods']))
    # soft_target = np.array([e[0] for e in soft_target.values])

    # w = df.evt_weight.values
    # # w = (w * (len(w) / w.sum())).values

    # n = Z.shape[0]
    # ix = range(n)
    # np.random.shuffle(ix)

    # log('determining train/test/val splits')
    # tr, te, val = ix[:(n / 2)], ix[(n / 2) : int((n / 2) + 0.18 * n)], ix[int((n / 2) + 0.18 * n):]

    # splits = {'train' : np.array(tr), 'test' :  np.array(te), 'validation' : np.array(val)}

    # data = {
    #     'Z' : Z,
    #     'y' : y,
    #     'soft_target' : soft_target,
    #     'weights' : w,
    #     'splits' : splits
    # }

    # log('saving to h5 archive...')
    # io.save(os.path.join(SAMPLE_PATH, 'MEM-db.h5'), data)






In [ ]:


datasetA = io.load('MEM-A-0500.h5')
datasetB = io.load('MEM-B-0500.h5')
datasetC = io.load('MEM-C-0500.h5')
datasets = [datasetA, datasetB, datasetC]
print 'datasetA', datasetA.keys()
print 'datasetB', datasetB.keys()
print 'datasetC', datasetC.keys()

np.random.seed(0)
print np.random.rand(1)

train =  [datasetA.items[np.random.choice(len(datasetA.values), size=5000, replace = False)]] 
print np.shape(test)
print np.shape(train)
               
test =  [datasetB[np.random.choice(len(datasetB), size=3000, replace = False)]] 
print np.shape(test)

eval =  [datasetC[np.random.choice(len(datasetC), size=2000, replace = False)]]
print np.shape(eval)

io.save('Samples_0500_train.h5', train)
io.save('Samples_0500_test.h5', test)
io.save('Samples_0500_eval.h5', eval)
